Foremost, Libraries.

In [ ]:
from random import randint
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.extensions import Initialize
from qiskit.quantum_info import random_statevector

The following functions allow us to easily switch between using a simulator or an actual system and get results.

In [ ]:
# Get a backend based on a type
def get_backend(backend_type):
    if backend_type == "simulator":
        return Aer.get_backend('aer_simulator')
    elif backend_type == "system":
        return IBMQ.load_account().get_backend('ibmq_manila')
    return None

# Get results of a Quantum Circuit
def get_results(backend, backend_type, qc):
    if backend_type == "simulator":
        qobj = assemble(transpile(qc, backend))
        return backend.run(qobj).result().get_counts()
    elif backend_type == "system":
        return execute(qc,backend,shots=1024).result().get_counts()
    return None

The following functions allow you to initalize a qubit in a random state. There is also a reverse function given a gate that allows you to return to whatever state you where previously in.

In [ ]:
# 1 Qubit Gate - Generate a random state
def random_state_gate():
    psi = random_statevector(2)
    gate = Initialize(psi)
    gate.label = "initializer"
    return gate

# N Qubit Gate - Reverse a gate
def reverse_gate(gate):
    inverse_gate = gate.gates_to_uncompute()
    inverse_gate.label = "disentangler"
    return inverse_gate

The following functions are designed to help with the Qunatum Digital Signature protocol.

In [ ]:
def create_private_key(length):
    return [ ("".join([str(randint(0, 1)) for i in range(2)])) for x in range(length * 2) ]

def print_private_key(priv_key):
    print(f"Private Key:")
    half = len(priv_key) // 2
    for i in range(half):
        print(f"{priv_key[i]} ", end='')
    print(end='\n')
    for i in range(half):
        print(f"{priv_key[i+half]} ", end='')

def generate_public_key(priv_key):
    qc = QuantumCircuit(len(priv_key))
    for idx, val in enumerate(priv_key):
        if val == "00":
            pass
        elif val == "01":
            qc.x(idx)
        elif val == "10":
            qc.h(idx)
        else:
            qc.x(idx)
            qc.h(idx)
    return qc.to_gate(label="public key")

def create_message(length):
    return [ ("".join([str(randint(0, 1)) for i in range(2)])) for x in range(length) ]

def print_msg_with_sig(msg, sig):
    print(f"Classical Message:")
    for i in range(len(msg)):
        print(f"{msg[i]} ", end='')
    print(end='\n')
    print(end='\n')
    print(f"Classical Signature:")
    for i in range(len(sig)):
        print(f"{sig[i]} ", end='')

def sign_message(msg, priv_key):
    sig = []
    for idx, val in enumerate(msg):
        if val == "00" or val == "11":
            sig.append(priv_key[idx])
        else:
            sig.append(priv_key[idx + (len(priv_key)//2)])
    return sig

def generate_quantum_signature(sig):
    qc = QuantumCircuit(len(sig))
    for idx, val in enumerate(sig):
        if val == "00":
            pass
        elif val == "01":
            qc.x(idx)
        elif val == "10":
            qc.h(idx)
        else:
            qc.x(idx)
            qc.h(idx)
    return qc.to_gate(label="quantum signature")

def swap_test_no_measure(qc, res, q1, q2):
    qc.h(res)
    qc.cswap(res, q1, q2)
    qc.h(res)

def signature_swap_test(msg, sig, pk, swap):
    qc = QuantumCircuit(pk, sig, swap)
    for idx, val in enumerate(msg):
        if val == "00" or val == "11":
            swap_test_no_measure(qc, swap[idx], pk[idx], sig[idx])
        else:
            swap_test_no_measure(qc, swap[idx], pk[idx+len(msg)], sig[idx])
    return qc.to_gate(label="swap test")

We now start our logic below.

In [ ]:
length = 6
matrix_length = length * 2
priv_key = create_private_key(length)

print_private_key(priv_key)

In [ ]:
pk = QuantumRegister(length*2, name="pk")
sig = QuantumRegister(length, name="sig")
swap = QuantumRegister(length, name="swap")
res = ClassicalRegister(length, name="result")
qc = QuantumCircuit(pk, sig, swap, res)

qc.append(generate_public_key(priv_key), pk)

In [ ]:
# TEMPORARY (JUST FOR PRINTING)
pk_temp = QuantumRegister(length*2, name="pk")
qc_temp = QuantumCircuit(pk_temp)
qc_temp.append(generate_public_key(priv_key), pk_temp)
plot_bloch_multivector(qc_temp)

In [ ]:
message = create_message(length)
signature = sign_message(message, priv_key)
print_msg_with_sig(message, signature)

We know send the message, signature, and quantum public key over from Alice to Bob.

In [ ]:
# AN ATTACKER CORRUPTED THE MESSAGE
#message = create_message(length)
#print_msg_with_sig(message, signature)

In [ ]:
qc.append(generate_quantum_signature(signature), sig)

In [ ]:
# TEMPORARY (JUST FOR PRINTING)
sig_temp = QuantumRegister(length, name="sig")
qc_temp = QuantumCircuit(sig_temp)
qc_temp.append(generate_quantum_signature(signature), sig_temp)
plot_bloch_multivector(qc_temp)

In [ ]:
qc.append(signature_swap_test(message, sig, pk, swap), range(length*4))
qc.measure(swap, res)
qc.draw()

In [ ]:
btype = "simulator"
backend = get_backend(btype)
results = get_results(backend, btype, qc)
plot_histogram(results)